In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from squlearn import Executor
from squlearn.feature_map import ChebPQC
from squlearn.expectation_operator import SummedPaulis
from squlearn.qnn import QNNRegressor, SquaredLoss
from squlearn.optimizers import Adam

In [ ]:
executor = Executor("statevector_simulator")

Define parameterized quantum circuit $Q(x,\theta)$

In [ ]:
nqubits = 4
number_of_layers = 3
pqc = ChebPQC(nqubits, 1, num_layers=number_of_layers, closed=False)
pqc.draw()

Definition of the cost operator:
$\hat{C}(\vartheta) = \vartheta_0 \hat{I} + \sum_i \vartheta_i \hat{Z}_i $

In [ ]:
cost_op = SummedPaulis(nqubits)
print(cost_op)

Example function: $log(x)$

In [ ]:
x_space = np.arange(0.1, 0.9, 0.01)
ref_values = np.log(x_space)

np.random.seed(13)
param_ini = np.random.rand(pqc.num_parameters) * 4
param_op_ini = np.ones(cost_op.num_parameters)

print(param_ini, param_op_ini)

Optimization of the regression

In [ ]:
ADAM = Adam({"maxiter_total": 100, "maxiter": 25, "lr": 0.1, "log_file": "adam_log.csv"})

reg = QNNRegressor(
    pqc,
    cost_op,
    executor,
    SquaredLoss(),
    ADAM,
    param_ini,
    param_op_ini,
    batch_size=10,
    epochs=30,
    shuffle=True,
)

reg.fit(x_space, ref_values)

In [ ]:
print("Result Param:", reg.param)
print("Result ParamOp:", reg.param_op)

In [ ]:
x = np.arange(np.min(x_space), np.max(x_space), 0.005)
y = reg.predict(x)
plt.plot(x, np.log(x))
plt.plot(x, y)